# Hybrid SUV Data Roll-Up

Link to the example in the Cookbook:\
[https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2[...]5/views/_18_0_6_e64033a_1490027434014_467163_56631](https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_2_17_7_33_25_AM_3ccfaf88_159fe0d7ba9__7d4e_cae_tw_jpl_nasa_gov_127_0_0_1/master/documents/_18_5_2_e64033a_1513278979739_556269_28755/views/_18_0_6_e64033a_1490027434014_467163_56631)


## Structure
![img/HSUV_structure.svg](img/HSUV_structure.svg)
## Data Roll-Up Pattern
![img/HSUV_pattern.svg](img/HSUV_pattern.svg)

In [4]:
package 'HybridSUV Dynamic Data Roll-Up'{
    import SI::Real;
    import ScalarFunctions::sum;
    import 'Data Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    
    value type bps specializes Real;
    
    package 'Data Roll-Up Pattern'{
       
        block 'Data Roll-up Aspect'{
            part subRollUp : 'Data Roll-up Aspect'[*];
            
            value 'totalDataRate' : bps = sum(subRollUp::totalDataRate) + behavior::dataRate;    
        
            value 'dataRateInStateOn' : bps;
            value 'dataRateInStateStandby' : bps;          

            exhibit behavior : 'Data Roll-up Behavior';
        } //END block 'Data Roll-up Aspect'
    } // END package
        
    
    package 'Structure'{
        block 'Body Control Module'  specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
        
        block 'Brake Control Module'  specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
        
        block 'Braking System' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part brakeControlModule : 'Brake Control Module' subsets subRollUp{
                value 'dataRateInStateOn' : bps redefines behavior::dataRateInStateOn = 10;
            }
        }
        
        block 'Power Control Unit' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part bodyControlModule : 'Body Control Module' subsets subRollUp;
            part powertrainControlModule : 'Powertrain Control Module' subsets subRollUp;
        }
        
        block 'Powertrain Control Module' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
        
        block 'HybridSUV' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part powerControlUnit : 'Power Control Unit' subsets subRollUp;
            part brakingSystem : 'Braking System' subsets subRollUp;
        }
 
    } // END package 'Structure'
    
    package Behavior{
        value type DStandbySignal;
        value type DTurnOnSignal;
        value type DTurnOffSignal;
        
        
        state def 'Data Roll-up Behavior' {
            value 'dataRate' : bps;
            
            entry; then Off;
            
            state Off{
                entry action{
                    dataRate = 0.0;
                }
            }
            transition off_to_standby
                first Off
                accept DStandbySignalthen StandBy;
            transition off_to_on
                first Off
                accept DTurnOnSignal then On;
            
            state StandBy{
                entry action{
                    dataRate = dataRateInStateStandby; // how to reference this VP?
                }
            }
            transition standby_to_off
                first StandBy
                accept DTurnOffSignal then Off;
            transition standby_to_on
                first StandBy
                accept DTurnOnSignal then On;
            
            state On{
                entry action{
                    dataRate = dataRateInStateOn; // how to reference this VP?
                }
            }
            transition on_to_standby
                first On
                accept DStandbySignal then StandBy;
            transition on_to_off
                first On
                accept DTurnOffSignal then Off;
        } // END state def 'Data Roll-up Behavior'
    } // END package Behavior

    package Configurations {
        // specify difference data configurations for analysis
    } // END package
    
    package 'Analysis' {
        block 'Test Driver' {
            perform action scenario {
                send DTurnOn to hsuv::powerControlUnit::bodyControlModule; // how reference hsuv?
                assert hsuv::powerControlUnit::bodyControlModule inState(On);
                send DStandBy to hsuv::powerControlUnit::brakeControlModule;
                assert hsuv::powerControlUnit::brakeControlModule inState(StandbBy);
                send DStandBy to hsuv::brakingSystem::bodyControlModule;
                assert hsuv::brakingSystem::bodyControlModule inState(StandbBy);
            } // END scenario
        } // END block
        
        block 'Analysis Context'{
            value total : bps;
            
            part hsuv : System::HybridSUV;
            part testDriver : 'Test Driver';
            
            value total_context : bps = hsuv::totalDataRate;
        } // END block
    } // END package 'Analysis'
} // END package 'HybridSUV Dynamic Data Roll-Up'


ERROR:mismatched input ';' expecting 'as' (4.sysml line : 20 column : 55)
ERROR:no viable alternative at input 'entry' (4.sysml line : 59 column : 13)
ERROR:no viable alternative at input 'transition' (4.sysml line : 66 column : 13)
ERROR:missing EOF at '}' ( line : 79 column : 13)
